In [2]:
import os
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
from astropy.coordinates import SkyCoord
import astropy.units as u

def deg2sex(ra_deg, dec_deg):
    # Create a SkyCoord object with the specified units
    coords = SkyCoord(ra=ra_deg * u.degree, dec=dec_deg * u.degree)
    # Extract the converted sexagesimal values
    ra_sexagesimal = coords.to_string(style='hmsdms', precision=2)
    return f"{ra_sexagesimal.split(' ')[0]}", f"{ra_sexagesimal.split(' ')[1]}"

# 1 convert the center coords to det XY

In [8]:
# convert the center coords to det XY

# read the fk5 eckert region
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')

ids = dat_f['SRCID']
# ids = ['SDSSTG40241']

for ID in tqdm(ids): 
    os.chdir(f'{rootdir}/{ID}/fk5_regs')
    for regfile in glob('R500_fk5.reg'):
        with open(regfile) as f:
            line = f.readlines()[-1]
            ra = line.split('(')[-1].split(',')[0]
            dec = line.split(',')[1].split()[0]

        # write the command to convert 
        
        with open('deg2det.sh', 'w') as f:
            f.write('source ../set_sas.txt\n')
            for name in ['mos1', 'mos2', 'pn']:
                f.write(f'''esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='../{name}-allevc-exchips.fits' 2>&1 | tee deg2det-{name}.log\n''')



100%|██████████| 45/45 [00:00<00:00, 3714.48it/s]


# 2 create the region files

2.1 define the shape and the radius of the region in XMM region format

In [14]:
#### define the shape and the radius of the region in XMM region format####

def xmm_circ(x, y, r):
    return f'&&((DETX,DETY) IN circle({x},{y},{r}))'
def xmm_annu(x, y, r1, r2):
    return f'&&((DETX,DETY) IN annulus({x},{y},{r1},{r2}))'
def xmm_elli(x, y, r1, r2, rot):
    return f'&&((DETX,DETY) IN ellipse({x},{y},{r1},{r2},{rot}))'

2.2 load the detxy of the center of the source

In [12]:
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dats/logNS'
dat_f = pd.read_csv(f'{datpath}/readme.csv')
savepath = f'{rootdir}/dom'
savefile = f'source_centers'
ids = dat_f['SRCID']
    
for name in ['mos1', 'mos2', 'pn']:
    op_dict = {'SRCID':ids, 'detX':np.zeros(len(ids)), 'detY':np.zeros(len(ids))}
    for i, ID in tqdm(enumerate(ids)): 
        os.chdir(f'{rootdir}/{ID}/fk5_regs')
        with open(f'deg2det-{name}.log') as f:
            for line in f:
                if '# detX' in line:
                    # Extract detX and detY values
                    op_dict['detX'][i], op_dict['detY'][i] = map(float, next(f).split())
    
    df = pd.DataFrame(op_dict)
    df.to_csv(f'{savepath}/{savefile}_{name}.csv')


45it [00:00, 6834.82it/s]
45it [00:00, 8495.84it/s]
45it [00:00, 7667.21it/s]


2.3 create the region files

In [18]:
#### basic parameters ####
am2xy = 60 * 20
bkg_rad = (12.1242,14.799) # arcmin

#### define paths ####
rootdir = '/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/'
datpath = f'{rootdir}/dom'


for name in ['mos1', 'mos2', 'pn']:
    # load the detxy of the center of the source
    dat_f = pd.read_csv(f'{datpath}/source_centers_{name}.csv')
    ids = dat_f['SRCID']
    for i, ID in tqdm(enumerate(ids)): 
        os.chdir(f'{rootdir}/{ID}/fk5_regs')
        ## bkg ##
        with open(f'{ID}-{name}_bkg.txt', 'w') as f:
            f.write(xmm_annu(dat_f['detX'][i], dat_f['detY'][i], np.round(bkg_rad[0] * am2xy, decimals=2), np.round(bkg_rad[1]* am2xy, decimals=2)))
        ## 0.1r500c ##
        # read radius from fk5 regfile
        with open(f'R500_01_fk5.reg') as f:
            line = f.readlines()[-1]
            rad = float(line.split(',')[-1].split(')')[0][:-1])

        with open(f'{ID}-{name}_R500_01.txt', 'w') as f:
            f.write(xmm_circ(dat_f['detX'][i], dat_f['detY'][i], np.round(rad * am2xy, decimals=2)))







45it [00:00, 2229.59it/s]
45it [00:00, 3632.62it/s]
45it [00:00, 4991.77it/s]
